In [38]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_datasets as tfds

# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [39]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [40]:
wandb.login()

True

In [41]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'kernel_size': {
          'values': ['3x3', '5x5']
        },
    'net_filter_size': {
          'values': [8, 16, 32]
        },
    'net_n': {
          'values': [3, 5, 8]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 2},
    'learning_rate': {
        'value': 0.000004},
    'batch_size': {
        'value': 64},
    })

In [42]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'batch_size': {'value': 64},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 2},
                'kernel_size': {'values': ['3x3', '5x5']},
                'learning_rate': {'value': 4e-06},
                'net_filter_size': {'values': [8, 16, 32]},
                'net_n': {'values': [3, 5, 8]}}}


In [43]:
sweep_id = wandb.sweep(sweep_config, project="TestBeforeMain")
# sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: wfks68fe
Sweep URL: https://wandb.ai/takim/TestBeforeMain/sweeps/wfks68fe


In [44]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Lambda, Add, Input, GlobalAveragePooling2D, Flatten, Dense, Softmax
from tensorflow.keras import Model

def ResidualBlock(x, filter_size, is_switch_block, kernel_size):

    # note that if is_switch_block true, it means that output will not be the same as the input
    # so while merging the residual connection, we need to adapt to it
    # this adaptation could be with a conv layer, or a simple downsampling + padding is enough.

    x_skip = x # save original input to the block

    if not is_switch_block:
        x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same')(x)
    else:
        x = Conv2D(filter_size, kernel_size=kernel_size, strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)

    if is_switch_block: # takes every second element to half(v) spatial dimension and then adds padding to each side for matching filter (last) dimension
        x_skip = Lambda(lambda x: tf.pad(x[:, ::2, ::2, :], tf.constant([[0, 0,], [0, 0], [0, 0], [filter_size//4, filter_size//4]]), mode="CONSTANT"))(x_skip)

    x = Add()([x, x_skip])
    x = ReLU()(x)

    return x

def ResidualBlocks(x, filter_size, n, kernel_size):
    for group in range(3): # a stack of 6n layers, 3×3 convolutions, feature maps of sizes {4fs, 2fs, fs}, 2n layers for each size
        for block in range(n):
            if group > 0 and block == 0: # double filter size
                filter_size *= 2
                is_switch_block = True
            else:
                is_switch_block = False
                
            x = ResidualBlock(x, filter_size, is_switch_block, kernel_size)

    return x

def create_model(config):

    filter_size = config['net_filter_size']
    n = config['net_n']
    kernel_size = (3, 3) if config['kernel_size'] == '3x3' else (5, 5)

    inputs = Input(shape=(32, 32, 3))
    x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = ResidualBlocks(x, filter_size, n, kernel_size)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(10)(x)
    outputs = Softmax()(x)

    model = Model(inputs, outputs, name=f"ResNet-{n*6+2}")
    return model

In [45]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        tf.keras.backend.clear_session()
        model = create_model(config)
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        history = model.fit(x_train, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )
        
        test_stats = model.evaluate(x_test, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [46]:
wandb.agent(sweep_id, train, count=1)
# wandb.agent(sweep_id, train)

wandb: Agent Starting Run: tor8qu6a with config:
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 2
wandb: 	kernel_size: 3x3
wandb: 	learning_rate: 4e-06
wandb: 	net_filter_size: 8
wandb: 	net_n: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/2
704/704 [==============================] - 14s 18ms/step - loss: 4.2435 - accuracy: 0.1006 - top@3_accuracy: 0.3329 - val_loss: 3.6877 - val_accuracy: 0.0950 - val_top@3_accuracy: 0.3422
Epoch 2/2
313/313 [==============================] - 2s 5ms/step - loss: 3.1254 - accuracy: 0.1105 - top@3_accuracy: 0.3795


best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁█
epoch/epoch,▁█
epoch/learning_rate,▁▁
epoch/loss,█▁
epoch/top@3_accuracy,▁█
epoch/val_accuracy,▁█
epoch/val_loss,█▁
epoch/val_top@3_accuracy,▁█
test_acc,▁
